In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns

import statsmodels.api as sm

from sklearn.model_selection import train_test_split, cross_val_score, cross_validate, KFold
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer, OneHotEncoder
from sklearn.impute import SimpleImputer

from sklearn.metrics import roc_auc_score, roc_curve

from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression


pd.set_option("display.max_columns",200)

def print_value_counts(df):
    for i in df.columns:
        print(f'column: {i}')
        display(df[i].value_counts())
        print()

Steps:

1. Select relevant cols
2. Impute NA - strategies: categorical - mode and NA, numeric median (or sth else)
3. one-hot encode categorical
4. merge categorical and numeric 
5. Fit models

Contents:

- load data
- train test split
- preprocess categorical vars
- preprocess numeric vars
- run CV

## Load data

In [ ]:
master_table = pd.read_pickle('data/master_table.pkl')

In [ ]:
master_table.head()

## EDA with filtering approach and feature selection

In [ ]:
master_table['year_of_birth'] = master_table['year_of_birth'].astype(float)

In [ ]:
cols_to_model = [
#  'accident_id',
 'lighting',
 'localization',
 'intersection_type',
 'weather',
 'collision_type',
#  'com',
#  'address',
#  'gps',
#  'lat',
#  'long',
#  'departament',
#  'time',
 'year',
 'month',
 'hour',
 'road_category',
 'road_regime',
 'no_lanes',
 'reserved_lane',
 'road_gradient',
 'road_plan',
 'road_condition',
 'infrastructure',
 'accident_situation',
#  'user_id',
 'place_in_car',
 'user_type',
#  'injury_type',
 'sex',
 'equipment_used',
 'pedestrian_action',
 'pedestrian_alone',
 'year_of_birth',
#  'vechicle_number',
#  'y'
]


In [ ]:
master_table[cols_to_model].dtypes

TODO:

- get dummies (with na)
- 0 variance
- m

### Categorical columns

In [ ]:
df_cat = master_table[cols_to_model].select_dtypes('object')
df_cat.head()

Replacing NA's - for now with 'NA'

In [ ]:
si = SimpleImputer(missing_values=np.nan, strategy='constant', fill_value = 'NA', add_indicator=False)
temp = si.fit_transform(df_cat)
df_cat2 = pd.DataFrame(temp, columns = df_cat.columns)
df_cat2.head()

One hot encoding

In [ ]:
df_cat3 = pd.get_dummies(df_cat2, drop_first = True)

In [ ]:
df_cat3.columns = ['d_' + col for col in list(df_cat3.columns)]

In [ ]:
df_cat_out = df_cat3.copy()

In [ ]:
df_cat_out.head()

In [ ]:
df_cat_out.shape

### Numeric columns 

In [ ]:
cols_num = [
 'year',
 'month',
 'hour',
 'year_of_birth'
]

In [ ]:
df_num = master_table[cols_num]
df_num.head()

In [ ]:
df_num.isna().mean()

Imputing median

In [ ]:
si = SimpleImputer(missing_values=np.nan, strategy='median', add_indicator=False)
temp = si.fit_transform(df_num)
df_num2 = pd.DataFrame(temp, columns = df_num.columns)
df_num2.head()

Basic feature engineering - age at the time of accident

In [ ]:
df_num2['user_age'] = df_num2['year'] - df_num2['year_of_birth']
df_num3 = df_num2.drop(['year_of_birth'], axis =1 )

In [ ]:
df_num_out = df_num3.copy()

In [ ]:
df_num_out.head()

### Merging categorical and numeric columns

In [ ]:
df_out = df_cat_out.join(df_num_out)

In [ ]:
df_out['y'] = master_table.y

In [ ]:
df_out.head()

## Feature selection - removing near-zero-variance predictors

In [ ]:
from sklearn.feature_selection import mutual_info_classif, mutual_info_regression, RFE, VarianceThreshold

In [ ]:
df_num_out.var()

Numerical variables are not near-zero-variance

In [ ]:
variances = []

for col in df_out.columns:
    variances.append(df_out[col].var())

In [ ]:
pd.DataFrame({'col': list(df_out.columns), 'var': variances}).sort_values(by = 'var').head(10)

For categorical variables - some have very low variance. A better way to filter these out is probably remove variables with very low percentage.

In [ ]:
df_out.select_dtypes('uint8').apply(lambda x: np.mean(x==1)).sort_values()

In [ ]:
perc_tre = 0.02

one_perc = df_out.select_dtypes('uint8').apply(lambda x: np.mean(x==1))
non_zero_var_cols = list(one_perc[one_perc > perc_tre][one_perc < 1 - perc_tre].index)

In [ ]:
non_zero_var_cols = non_zero_var_cols + list(df_num_out.columns)

In [ ]:
df_out2 = df_out[non_zero_var_cols + ['y']]

In [ ]:
df_out2.head()

## Feature selection - mutual information

In [ ]:
minfos_cat = []

In [ ]:
for col in df_out2.select_dtypes('uint8').columns:
#     print(col)
    minfos_cat.append(mutual_info_classif(df_out2[col].values.reshape(-1,1),
                                                        df_out2["y"].values, discrete_features = True)[0])

In [ ]:
minfos2= [(col, val) for col, val in zip(df_out2.select_dtypes('uint8').columns, minfos_cat)]

In [ ]:
pd.DataFrame(minfos2, columns = ['column', 'mutual_info']).sort_values(by='mutual_info', ascending=False).head()

Don't run below - takes forever and brakes everything

In [ ]:
# minfos_num = []
# for col in df_out2.select_dtypes('float64').columns:
#     print(col)
#     minfos_num.append(mutual_info_regression(df_out2[col].values.reshape(-1,1),
#                                                         df_out2["y"].values))

## Train test split

For each accident in the data, there can be multiple users engaged. This means that we should control for data leakage. People from one accident should be placed together in either train or test dataset. One way to achieve this and simultanously keep distribution of y is to sample accident_id, and from that obtain train test split.


In [ ]:
master_table['accident_id']

In [ ]:
df_out2['accident_id'] = master_table['accident_id']#.groupby('y').accident_id.count()

Test what proportion of y we need in both train and test datasets

In [ ]:
df_out2.y.value_counts(normalize = True)

Around 23% of y is equal to 1 (seriously injured or killed).
Randomly select 30% of accident_id and check the distribution of y obtained:

In [ ]:
temp = pd.DataFrame({'test_id': df_out2.accident_id.sample(frac = 0.3), 'if_test': 1}).set_index('test_id')

temp2 = df_out2.set_index('accident_id').join(temp)
temp3 = temp2.reset_index().rename({'index': 'accident_id'}, axis = 1)
temp3.head()

Finally, create training and test sets

In [ ]:
test = temp3.query('if_test == 1').drop('if_test', axis =1)
train = temp3.query('if_test.isna()').drop('if_test', axis =1)

Checking if procedure was correct:

In [ ]:
print(test.shape)
print(train.shape)

print(test.y.mean())
print(train.y.mean())

In [ ]:
x_train = train.drop(['y', 'accident_id'], axis = 1)
x_test = test.drop(['y', 'accident_id'], axis = 1)

y_train = train[['y']]
y_test = test[['y']]

In [ ]:
x_train.head()

In [ ]:
# x_train, x_test, y_train, y_test = train_test_split(df_out2.drop('y', axis=1), 
#                                                     df_out2.y, 
#                                                     stratify=df_out2.y, 
#                                                     test_size=0.3, 
#                                                     random_state = 3)

### Modeling

KNN is slooow

In [ ]:
# model = KNeighborsClassifier()
# a = cross_validate(model, df_out, y_train, cv=3, scoring = 'roc_auc', verbose = 1)

In [ ]:
model1 = LogisticRegression(n_jobs=-1, verbose=2)
a = cross_validate(model1, x_train, y_train, cv=2, scoring = 'roc_auc', verbose = 1, n_jobs=-1)

In [ ]:
fitted = model1.fit(x_train, y_train)

In [ ]:
roc_auc_score(y_train, fitted.predict_proba(x_train)[:,1])


## Stepwise feature selection

In [ ]:
from sklearn.svm import LinearSVC

In [ ]:
model2 = LinearSVC()

In [ ]:
model2.fit(x_train, y_train)

In [ ]:
from sklearn.feature_selection import RFECV, RFE

In [ ]:
rfe = RFE(model2, n_features_to_select = 1)

In [ ]:
rfe.fit(x_train, y_train)